In [1]:
import numpy as np
import cvxpy as cp
from qutip import Qobj, tensor, ptrace, qeye, num, ket2dm
from numpy.polynomial.hermite import hermgauss, Hermite
from math import factorial
from itertools import zip_longest
import time
from tqdm import tqdm  # 添加进度条

In [2]:
def cv_state_S26(r, eta1, eta2, max_p):
    
    t = np.tanh(r)
    sqrt_eta1, sqrt_1_e1 = np.sqrt(eta1), np.sqrt(1-eta1)
    d = max_p+1
    amps = {}  # map (nA,nB,nC)->complex amplitude

    fac = factorial
    for p in range(max_p+1):
        pref_p = (t**p) / ((2**p) * np.cosh(r))
        for k1 in range(p+1):
            f_k1 = (2*eta1-1)**k1
            for k2 in range(k1+1):
                f_bs1 = (4*sqrt_eta1*sqrt_1_e1)**(p-k1)
                num1 = fac(p + 2*k2 - k1) * fac(p - (2*k2 - k1))
                den1 = fac(p-k1)*fac(k1-k2)*fac(k2)
                sqrt1 = np.sqrt(num1/den1)

                nA = p + (2*k2 - k1)
                nB1= p - (2*k2 - k1)

                for k3 in range(nB1+1):
                    # binomial factor & second-BS sqrt
                    c = fac(nB1) / (fac(k3)*fac(nB1-k3))
                    sqrt2 = np.sqrt(c * eta2**k3 * (1-eta2)**(nB1-k3))

                    sign = (-1)**(p - (3*k2 + k3))
                    nB, nC = k3, nB1-k3

                    amp = pref_p * f_bs1 * f_k1 * sqrt1 * sqrt2 * sign
                    amps[(nA,nB,nC)] = amps.get((nA,nB,nC),0) + amp

    vec = []
    for nA in range(d):
      for nB in range(d):
        for nC in range(d):
          vec.append(amps.get((nA,nB,nC),0.0))
    psi = Qobj(np.array(vec), dims=[[d,d,d],[1,1,1]]).unit()
    return psi

In [3]:
def make_homodyne_povm(N_bins, T, cutoff, Nq=200):
    d = cutoff + 1

    # Gauss–Hermite nodes/weights
    x, w = hermgauss(Nq)

    # Build Φ_n(x_i) = norm_n * H_n(x_i) 
    Phi = np.empty((d, Nq))
    for n in range(d):
        Hn   = Hermite.basis(n)  # physicists' H_n
        norm = 1.0 / (np.pi**0.25 * np.sqrt(2.0**n * factorial(n)))
        Phi[n, :] = norm * Hn(x)

    # Periodic wrap and bin index (each node lands in exactly one bin)
    tri    = T / N_bins
    xmod = x - np.floor(x / T) * T          # in [0, T)
    idx  = np.floor(xmod / tri).astype(int)
    idx  = np.clip(idx, 0, N_bins - 1)

    Ms = []
    for b in range(N_bins):
        mask = (idx == b).astype(float)     # shape (Nq,)
        W    = w * mask
        mat  = (Phi * W) @ Phi.T            # (d,Nq)*(Nq,) -> (d,Nq); @ Phi.T -> (d,d)
        Ms.append(Qobj(mat, dims=[[d],[d]]))

    # Completeness: Σ_b M_b ≈ I
    I = sum(M.full() for M in Ms)
    # 降低断言的严格性，避免数值误差导致的错误
    assert np.allclose(I, np.eye(d), atol=1e-6), "Σ_b M_b ≠ I; try larger Nq."
    return Ms

In [4]:
def certify_randomness_ns(psi, Mbases, Mcases, solver_params=None):
    """
    优化版的随机性认证函数
    solver_params: 求解器参数字典，可以覆盖默认参数
    """
    d = psi.dims[0][0]           # local Hilbert dim (should be cutoff+1)
    rho = ket2dm(psi)            # make it a density operator
    nB = len(Mbases['x'])  
    nC = len(Mcases['x']) 

    # 1) assemblage
    sigma_obs = {}
    for y in ['x','p']:
        for z in ['x','p']:
            for b in range(nB):
                for c in range(nC):
                     Mb, Mc = Mbases[y][b], Mcases[z][c]
                     op = tensor(qeye(d), Mb, Mc)              # I ⊗ Mb ⊗ Mc
                     sigma_obs[(b,c,y,z)] = ptrace(op * rho, [0])  # on A

    # 2) variables
    sigma = {}
    for e in range(nB):
        for e2 in range(nC):
            for y in ['x','p']:
                for z in ['x','p']:
                    for b in range(nB):
                        for c in range(nC):
                            sigma[(e,e2,b,c,y,z)] = cp.Variable((d,d), hermitian=True)

    constraints = []

    # 3) equality to observed assemblage
    for (b,c,y,z), obs in sigma_obs.items():
        lhs = sum(sigma[(e,e2,b,c,y,z)] for e in range(nB) for e2 in range(nC))
        constraints.append(lhs == cp.Constant(obs.full()))

    # 4) no-signaling per (B6)
    for e in range(nB):
        for e2 in range(nC):
        # C marginal independent of y
            for c in range(nC):
                  for z in ['x','p']:
                        lhs1 = sum(sigma[(e,e2,b,c,'x',z)] for b in range(nB))
                        lhs2 = sum(sigma[(e,e2,b,c,'p',z)] for b in range(nB))
                        constraints.append(lhs1 == lhs2)
        # B marginal independent of z
            for b in range(nB):
                  for y in ['x','p']:
                        lhs1 = sum(sigma[(e,e2,b,c,y,'x')] for c in range(nC))
                        lhs2 = sum(sigma[(e,e2,b,c,y,'p')] for c in range(nC))
                        constraints.append(lhs1 == lhs2)

    # 5) PSD
    for var in sigma.values():
        constraints.append(var >> 0)

    # 6) objective
    obj_expr = cp.sum([
        cp.real(cp.trace(sigma[(e,e2,e,e2,'x','x')]))
        for e in range(nB) for e2 in range(nC)
   ])
    prob = cp.Problem(cp.Maximize(obj_expr), constraints)

    # 默认求解器参数
    default_params = {
        'solver': cp.SCS,
        'verbose': False,
        'max_iters': 5000,  # 减少迭代次数，提高速度
        'eps': 1e-3,        # 放宽精度要求，提高速度
        'acceleration_lookback': 10,  # 添加加速策略
        'rho_x': 1e-3       # 调整rho参数以提高收敛性
    }

    # 合并用户提供的参数
    if solver_params:
        default_params.update(solver_params)
    
    try:
        start_time = time.time()
        result = prob.solve(**default_params)
        solve_time = time.time() - start_time
        
        P_g  = float(prob.value)                   # optimal guessing prob
        P_g  = float(np.clip(P_g, 0.0, 1.0))       # guard against tiny numerical drift
        Hmin = 0.0 if P_g <= 0 else max(0.0, -np.log2(P_g))
        
        # 如果求解精度不够，给出警告但继续执行
        if prob.status != cp.OPTIMAL:
            print(f"警告：求解状态为 {prob.status}，结果可能不够准确")
        
        return Hmin, P_g, solve_time
    except Exception as e:
        print(f"求解器错误: {e}")
        # 出现错误时返回默认值，避免程序中断
        return 0.0, 1.0, 0.0


In [5]:
if __name__ == "__main__":
    # 优化版参数 - 减少计算量
    r      = 0.345
    eta1   = 0.5
    p_max  = 1          
    cutoff = 1         
    d      = cutoff + 1
    N_bins = 4
    # 减少搜索点数量，从9个减少到5个
    T_B_vals = np.linspace(2, 10, 5)   # search Bob's Tx
    T_C_vals = np.linspace(2, 10, 5)   # search Charlie's Tx
    Nq     = 120  # 减少高斯-埃尔米特点数量，提高速度
    
    R_pi2 = (-1j * (np.pi/2) * num(d)).expm()
    
    Hmins = []
    total_time = 0  # 记录总计算时间
    
    # 使用tqdm添加进度条
    eta2_values = np.linspace(0, 1, 11)
    for i, eta2 in enumerate(tqdm(eta2_values, desc="处理eta2参数")):
        best_H = 0.0
        
        # 生成量子态
        psi = cv_state_S26(r, eta1, eta2, p_max)
        
        # 减少嵌套循环的深度 - 提前判断是否有希望得到更好的结果
        # 记录已处理的组合数
        combinations_processed = 0
        max_combinations = len(T_B_vals) * len(T_C_vals)
        early_stop_threshold = max_combinations * 0.7  # 70%组合后如果没有改进可以提前停止
        last_improvement = 0
        
        for TxB in T_B_vals:
            TpB = 2*np.pi*N_bins / TxB
            
            # Bob POVMs
            Mb_x = make_homodyne_povm(N_bins, TxB, cutoff, Nq=Nq)
            Mb_p_xform = make_homodyne_povm(N_bins, TpB, cutoff, Nq=Nq)
            Mb_p = [R_pi2.dag() * M * R_pi2 for M in Mb_p_xform]
            
            for TxC in T_C_vals:
                combinations_processed += 1
                
                # 早停机制：如果长时间没有改进且已处理大部分组合，提前停止
                if combinations_processed > last_improvement + early_stop_threshold:
                    print(f"  早停: eta2={eta2:.2f}，长时间无改进")
                    break
                
                TpC = 2*np.pi*N_bins / TxC
                
                # Charlie POVMs
                Mc_x = make_homodyne_povm(N_bins, TxC, cutoff, Nq=Nq)
                Mc_p_xform = make_homodyne_povm(N_bins, TpC, cutoff, Nq=Nq)
                Mc_p = [R_pi2.dag() * M * R_pi2 for M in Mc_p_xform]
                
                # 使用优化的求解器参数
                solver_params = {
                    'eps': 1e-3,  # 放宽精度，提高速度
                    'max_iters': 5000  # 减少最大迭代次数
                }
                
                # 对于中间值可以使用更激进的参数，两端值可以更精确
                if i in [0, len(eta2_values)-1]:
                    solver_params['eps'] = 1e-4  # 两端点使用更高精度
                
                Hmin, Pg, solve_time = certify_randomness_ns(
                    psi,
                    {'x': Mb_x, 'p': Mb_p},
                    {'x': Mc_x, 'p': Mc_p},
                    solver_params=solver_params
                )
                
                total_time += solve_time
                
                # 更新最佳值和最后改进的位置
                if Hmin > best_H:
                    best_H = Hmin
                    last_improvement = combinations_processed
                    
        Hmins.append(best_H)
        # 打印进度信息
        print(f"eta2={eta2:.2f}: 最佳H_min={best_H:.5f}")
    
    print(f"\n总计算时间: {total_time:.2f} 秒")
    print(f"最终Hmins结果: {Hmins}")

处理eta2参数:   9%|▉         | 1/11 [02:54<29:06, 174.63s/it]

  早停: eta2=0.00，长时间无改进
eta2=0.00: 最佳H_min=0.15576


处理eta2参数:  18%|█▊        | 2/11 [05:23<23:57, 159.76s/it]

eta2=0.10: 最佳H_min=0.00732


处理eta2参数:  27%|██▋       | 3/11 [07:49<20:27, 153.48s/it]

...

eta2=0.80: 最佳H_min=0.00657


处理eta2参数:  91%|█████████ | 10/11 [24:35<02:25, 145.02s/it]

eta2=0.90: 最佳H_min=0.00732


处理eta2参数: 100%|██████████| 11/11 [28:10<00:00, 153.67s/it]

eta2=1.00: 最佳H_min=0.15576

总计算时间: 1641.30 秒
最终Hmins结果: [np.float64(0.1557629557794112), np.float64(0.007320450433710241), np.float64(0.006567463440057703), np.float64(0.006184966129671333), np.float64(0.006324025396373586), np.float64(0.0063481189180133995), np.float64(0.006324025403448868), np.float64(0.0061849661317877605), np.float64(0.006567463438641764), np.float64(0.007320450429592538), np.float64(0.1557629557857033)]
